<a href="https://colab.research.google.com/github/ssgantayat/AIML-Programs/blob/main/Lab04_USA_Housing_Machine_Learning(Linear_Regression).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = ':https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F7039%2F10115%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240830%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240830T041708Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D0d0c588665a1b9d566119bd30d063696befede50443249d3e8de5290e376099a4c59f29cc466bea9a01ab2daee55291751cbc42531fdb6d87f3fa4f4606a7805edba2d4db7e8891e1848bf385a82ffd55836d99d45996981571a913aa13783264a61b512c62ea43152154e7e27928fdcafe9b048c95c5bb99ae88958d41490327db267ad7cbd1cd1c76761abb0100f638c2cd7f811c3bd8a59a310ee1f596cf3bfd4a460568541f2515b5d41030f6ecc1ec35e43906bc658b37b63d8530b3a35988a08bc5454398fcf99bf414e44721537e6a09117d361b425b700f7ed571aa21377024a1fd619f84bb7f792843e44cb48d4d6564ed1128c42e7d5cf18206194'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 371432 bytes downloaded
Downloaded and uncompressed: 
Data source import complete.


## **Predicting Housing Prices for regions in the USA.**

The data contains the following columns:

* 'Avg. Area Income': Avg. Income of residents of the city house is located in.
* 'Avg. Area House Age': Avg Age of Houses in same city
* 'Avg. Area Number of Rooms': Avg Number of Rooms for Houses in same city
* 'Avg. Area Number of Bedrooms': Avg Number of Bedrooms for Houses in same city
* 'Area Population': Population of city house is located in
* 'Price': Price that the house sold at
* 'Address': Address for the house

In [ ]:
# importing librarires
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline
#sns.set_style('whitegrid')


In [ ]:
# loading csv data to dataframe
USA_Housing = pd.read_csv('../input/USA_Housing.csv')
# checking out the Data
USA_Housing.head()


In [ ]:
#checking columns and total records
USA_Housing.info()

 **Generating descriptive statistics that summarize the central tendency, dispersion and shape of a dataset’s distribution, excluding NaN value.
**

In [ ]:
USA_Housing.describe()

## **Exploratory Data Analysis**


** Visualizing the Entire Data using pairplot **

Exploring types of relationship across the entire dataset

In [ ]:
sns.pairplot(USA_Housing)

## **Distribution of Housing Price Across USA**

In [ ]:
sns.distplot(USA_Housing['Price'],hist_kws=dict(edgecolor="black", linewidth=1),color='Blue')

In [ ]:
#Displaying correlation among all the columns
USA_Housing.corr()

## **Displaying correlation among all the columns using Heat Map**


In [ ]:
sns.heatmap(USA_Housing.corr(), annot = True)

## **Training a Linear Regression Model**

[](http://)We will need to first split up our data into an X array that contains the features to train on, and a y array with the target variable, in this case the Price column. We will toss out the Address column because it only has text info that the linear regression model can't use.

### X and y arrays

In [ ]:
#Getting all Coulmn names
USA_Housing.columns

In [ ]:
# Columns as Features
X = USA_Housing[['Avg. Area Income', 'Avg. Area House Age', 'Avg. Area Number of Rooms',
       'Avg. Area Number of Bedrooms', 'Area Population']]

In [ ]:
# Price is my Target Variable, what we trying to predict
y = USA_Housing['Price']

## **Training the Model**

Now that we've explored the data a bit, let's go ahead and split the data into training and testing sets.


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=101)

In [ ]:
#importing the Linear Regression Algorithm
from sklearn.linear_model import LinearRegression

In [ ]:
#creating LinearRegression Object
lm = LinearRegression()

In [ ]:
#Training the Data Model
lm.fit(X_train, y_train)

 ## **Model Evaluation**


Let's evaluate the model by checking out it's coefficients

In [ ]:
#Displaying the Intercept
print(lm.intercept_)

In [ ]:
coeff_df = pd.DataFrame(lm.coef_, X.columns, columns=['Coefficient'])
coeff_df

**Interpreting the coefficients:
**
- Holding all other features fixed, a 1 unit increase in **Avg. Area Income** is associated with an **increase of \$21.52 **.
- Holding all other features fixed, a 1 unit increase in **Avg. Area House Age** is associated with an **increase of \$164883.28 **.
- Holding all other features fixed, a 1 unit increase in **Avg. Area Number of Rooms** is associated with an **increase of \$122368.67 **.
- Holding all other features fixed, a 1 unit increase in **Avg. Area Number of Bedrooms** is associated with an **increase of \$2233.80 **.
- Holding all other features fixed, a 1 unit increase in **Area Population** is associated with an **increase of \$15.15 **.


## **Predictions from our Model**

Let's grab predictions off our test set and see how well it did!

In [ ]:
predictions = lm.predict(X_test)

In [ ]:
plt.scatter(y_test, predictions, edgecolor='black')

## **Residual Histogram**

**Ploting a histogram of the residuals and making sure it looks normally distributed. **

In [ ]:
sns.distplot((y_test - predictions), bins = 50, hist_kws=dict(edgecolor="black", linewidth=1),color='Blue')

** Calculating the Mean Absolute Error, Mean Squared Error, and the Root Mean Squared Error **

- **MAE** is the easiest to understand, because it's the average error.
- **MSE** is more popular than MAE, because MSE "punishes" larger errors, which tends to be useful in the real world.
- **RMSE** is even more popular than MSE, because RMSE is interpretable in the "y" units.

All of these are **loss functions**, because we want to minimize them.

In [ ]:
from sklearn import metrics

In [ ]:
print('MAE:', metrics.mean_absolute_error(y_test, predictions))
print('MSE:', metrics.mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))